In [1]:
import torch as T
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tensorflow.contrib.learn.python.learn.datasets.mnist import extract_images, extract_labels

In [2]:
class AI(nn.Module):
    def __init__(self,conv_kernel):
        super(AI, self).__init__()
        self.conv_kernel = conv_kernel
        self.lstm_layers = 1
        self.lstm_depth = 1000
        self.cnn1 = nn.Conv2d(1,self.conv_kernel,5,stride=2,padding=1)
        self.cnn1_2 = nn.Conv2d(self.conv_kernel,self.conv_kernel,3,stride=1)
        self.cnn2 = nn.Conv2d(self.conv_kernel,self.conv_kernel,3,stride=2,padding=0)
        self.rnn = nn.LSTM(5 * 5, self.lstm_depth,self.lstm_layers)
        self.fc1 = nn.Linear(self.conv_kernel * self.lstm_depth,self.lstm_depth)
        self.fc2 = nn.Linear(self.lstm_depth,10)
        self.out = nn.Dropout(0.9)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')     
        self.to(self.device)
    def forward(self,x):
        hx = T.zeros(self.lstm_layers,self.conv_kernel, self.lstm_depth).to(self.device)
        cx = T.zeros(self.lstm_layers,self.conv_kernel, self.lstm_depth).to(self.device)
        x = F.relu(self.cnn1(x))
        x = F.relu(self.cnn1_2(x))
        x = F.relu(self.cnn2(x))
        x = x.view(-1,self.conv_kernel,5 * 5)
        hx, cx = self.rnn(x, (hx, cx))
        x = hx.view(-1,self.conv_kernel * self.lstm_depth)
        x = T.tanh(self.fc1(x))
        x = T.sigmoid(self.fc2(x))
#        print(o.shape)
#        o = self.out(o)
        return x

In [3]:
class PLAYFIELD():
    def __init__(self,width,height):
        self.width = width
        self.height = height
        self.pf = np.zeros((self.width,self.height,2))
    def setStone(self,x,y,stone):
        s = np.zeros(2)
        s[stone] = 1
        self.pf[y][x] = s
    def get(self):
        return self.pf

In [4]:
with open('train-images-idx3-ubyte.gz', 'rb') as f:
    train_images = extract_images(f)
with open('train-labels-idx1-ubyte.gz', 'rb') as f:
    train_labels = extract_labels(f)

with open('t10k-images-idx3-ubyte.gz', 'rb') as f:
    test_images = extract_images(f)
with open('t10k-labels-idx1-ubyte.gz', 'rb') as f:
    test_labels = extract_labels(f)

Instructions for updating:
Please use tf.data to implement this functionality.
Extracting train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting train-labels-idx1-ubyte.gz
Extracting t10k-images-idx3-ubyte.gz
Extracting t10k-labels-idx1-ubyte.gz


In [5]:

ai_conv_kernel = 128
ai = AI(ai_conv_kernel)

max_mem = 1000
a = 28

inp = []
for i in range(max_mem):
    inp.append(np.random.randn(1, a, 28))
    
criterion = nn.MSELoss()#SmoothL1Loss()MSELoss
optimizer = optim.RMSprop(ai.parameters(), lr=0.00005)

losses = 0
for j in range(1):
    for i in range(60000):#train_images.shape[0]):
        def closure():
            optimizer.zero_grad()
            inp1 = train_images[i]#inp[i % max_mem].copy() + np.random.randn(1, a, 28) * 0.1
            inp1.shape = (1,1,28,28)
            inp1 = T.from_numpy(inp1).float().to(ai.device)
            out = ai(inp1)
    #        print(out)
    #        print(target)
#            target = out.cpu().detach().numpy().copy()
#            print(target.shape)
#            target[0][8 - 1] = np.zeros(10)
#            target[0][8 - 1][train_labels[i]] = 1.0

#            target[0] = np.zeros([1,8,10])
            target = np.zeros(10)
            target[train_labels[i]] = 1.0
#            for o in range(ai_conv_kernel):
#                oo = (o + 1) / ai_conv_kernel
#                oo = oo * oo
#                target[0][o] = oo * n + (1.0 - oo) * target[0][o]
#            target[0][8 - 1][train_labels[i]] = 1.0
#            print(target)
            target = T.from_numpy(np.array(target)).to(ai.device)
            loss = criterion(out, target.float())
            loss.backward()
            return loss
        loss = optimizer.step(closure)
        losses = losses + loss.item()
        if (i % 100) == 0:
            print('Step: ' + str(i) + ' loss: ' + str(losses / 100))
            losses = 0

Step: 0 loss: 0.002555728256702423
Step: 100 loss: 0.0659104516170919
Step: 200 loss: 0.03750291411415674
Step: 300 loss: 0.03371536461636424
Step: 400 loss: 0.025494956957409158
Step: 500 loss: 0.02888321423874004
Step: 600 loss: 0.023229636772011873
Step: 700 loss: 0.02121426889265422
Step: 800 loss: 0.016647743974026526
Step: 900 loss: 0.023323459564708174
Step: 1000 loss: 0.017756924685527337
Step: 1100 loss: 0.022209286977085865
Step: 1200 loss: 0.020750521204972755
Step: 1300 loss: 0.019313499630370642
Step: 1400 loss: 0.017268661420457646
Step: 1500 loss: 0.010340712676770636
Step: 1600 loss: 0.014461592787884002
Step: 1700 loss: 0.01070790683676023
Step: 1800 loss: 0.005780514274047164
Step: 1900 loss: 0.009259243803899152
Step: 2000 loss: 0.013625237210035265
Step: 2100 loss: 0.014043649158338667
Step: 2200 loss: 0.0074955815706562135
Step: 2300 loss: 0.00601973836272009
Step: 2400 loss: 0.007971899815129291
Step: 2500 loss: 0.011990603665872186
Step: 2600 loss: 0.006148691893

Step: 21300 loss: 0.0013352721837964054
Step: 21400 loss: 0.0010247543468153708
Step: 21500 loss: 0.0028146081238310215
Step: 21600 loss: 0.001695754688640818
Step: 21700 loss: 0.005052084001879393
Step: 21800 loss: 0.0011780223243169985
Step: 21900 loss: 0.0020647118575254806
Step: 22000 loss: 0.0057001873093895485
Step: 22100 loss: 0.0013549315787633987
Step: 22200 loss: 0.005628394633305334
Step: 22300 loss: 0.004978525052289342
Step: 22400 loss: 0.002418164162336325
Step: 22500 loss: 0.004431243413483799
Step: 22600 loss: 0.004772911886655038
Step: 22700 loss: 0.0036984093251908236
Step: 22800 loss: 0.0040301870169503215
Step: 22900 loss: 0.003052846887603664
Step: 23000 loss: 0.00027257765454123727
Step: 23100 loss: 0.004465306606211925
Step: 23200 loss: 0.002775338232058857
Step: 23300 loss: 0.0008886950139412875
Step: 23400 loss: 0.0035096258180249153
Step: 23500 loss: 0.0050866829056127475
Step: 23600 loss: 0.0029627092353007355
Step: 23700 loss: 0.004564551393305578
Step: 2380

Step: 42100 loss: 0.002025233378104332
Step: 42200 loss: 0.001971899538546893
Step: 42300 loss: 0.0009507109739570296
Step: 42400 loss: 0.0019003577536736316
Step: 42500 loss: 0.007341923468500369
Step: 42600 loss: 0.003841564080692308
Step: 42700 loss: 0.0010812711903294314
Step: 42800 loss: 0.002277383367111554
Step: 42900 loss: 0.005512177790427586
Step: 43000 loss: 0.001761825229613976
Step: 43100 loss: 0.0019925987418157832
Step: 43200 loss: 0.004688686827982451
Step: 43300 loss: 0.0043935747621520705
Step: 43400 loss: 0.000564281762143537
Step: 43500 loss: 0.002644684177989802
Step: 43600 loss: 0.003434341382094459
Step: 43700 loss: 0.0033344997667310848
Step: 43800 loss: 0.0020147325061327594
Step: 43900 loss: 0.0031451268388616427
Step: 44000 loss: 0.0018334619241092299
Step: 44100 loss: 0.003557969974601178
Step: 44200 loss: 0.0013908005270622682
Step: 44300 loss: 0.002705372204034404
Step: 44400 loss: 0.00162263286229134
Step: 44500 loss: 0.004596065963088876
Step: 44600 loss

KeyboardInterrupt: 

In [ ]:
count = 0
for i in range(10000):#train_images.shape[0]):
    optimizer.zero_grad()
    inp1 = test_images[i]#inp[i % max_mem].copy()
    inp1.shape = (1,1,28,28)
    inp1 = T.from_numpy(inp1).float().to(ai.device)
    out = ai(inp1).cpu().detach().numpy()
    m = np.argmax(out)
    if m == test_labels[i]:
#        print(m)
        count = count + 1
print("correct: " + str(count / 10000))#train_images.shape[0]))